# Stocks

In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import ig

## API Setup

In [8]:
details = ig.get_api('prod')
api = ig.APIHandler(details['url'], details['api_key'], details['user_name'], details['password'])
del details

In [10]:
# api.markets(93334)
print(api.prices('IX.D.FTSE.DAILY.IP', resolution='DAY',
                 from_date='2020-08-13 09:00:00', to_date='2020-08-14 11:00:00'))

https://api.ig.com/gateway/deal/prices/IX.D.FTSE.DAILY.IP/DAY/2020-08-13%2009%3A00%3A00/2020-08-14%2011%3A00%3A00
[{'prices': [{'snapshotTime': '2020/08/14 00:00:00', 'openPrice': {'bid': 6188.8, 'ask': 6192.8, 'lastTraded': None}, 'closePrice': {'bid': 6076.5, 'ask': 6080.5, 'lastTraded': None}, 'highPrice': {'bid': 6201.8, 'ask': 6203.8, 'lastTraded': None}, 'lowPrice': {'bid': 6033.2, 'ask': 6034.2, 'lastTraded': None}, 'lastTradedVolume': 91047}], 'instrumentType': 'INDICES', 'allowance': {'remainingAllowance': 9989, 'totalAllowance': 10000, 'allowanceExpiry': 527003}}]
